In [ ]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
text_df = pd.read_csv("/content/fake_or_real_news (1).csv", quoting=3, on_bad_lines='warn')
text = list(text_df.text.values)


joined_text = " ".join([str(item) for item in text])


with open("joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)


<ipython-input-2-afd786846f25>:1: ParserWarning: Skipping line 36: expected 5 fields, saw 6
Skipping line 51: expected 5 fields, saw 6
Skipping line 96: expected 5 fields, saw 8
Skipping line 97: expected 5 fields, saw 6
Skipping line 98: expected 5 fields, saw 12
Skipping line 101: expected 5 fields, saw 7
Skipping line 102: expected 5 fields, saw 13
Skipping line 103: expected 5 fields, saw 8
Skipping line 104: expected 5 fields, saw 6
Skipping line 106: expected 5 fields, saw 16
Skipping line 108: expected 5 fields, saw 6
Skipping line 109: expected 5 fields, saw 15
Skipping line 110: expected 5 fields, saw 11
Skipping line 113: expected 5 fields, saw 12
Skipping line 114: expected 5 fields, saw 11
Skipping line 116: expected 5 fields, saw 8
Skipping line 117: expected 5 fields, saw 6
Skipping line 118: expected 5 fields, saw 8
Skipping line 119: expected 5 fields, saw 9
Skipping line 120: expected 5 fields, saw 8
Skipping line 121: expected 5 fields, saw 7
Skipping line 129: expect

In [ ]:
partial_text = joined_text[:1000000]

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [ ]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [ ]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)

In [ ]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 167s 514ms/step - accuracy: 0.3732 - loss: 3.8725
Epoch 2/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 199s 503ms/step - accuracy: 0.4020 - loss: 3.6265
Epoch 3/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 157s 494ms/step - accuracy: 0.4262 - loss: 3.4060
Epoch 4/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 206s 505ms/step - accuracy: 0.4579 - loss: 3.1856
Epoch 5/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 203s 509ms/step - accuracy: 0.4910 - loss: 2.9682
Epoch 6/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 204s 514ms/step - accuracy: 0.5184 - loss: 2.7941
Epoch 7/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 204s 519ms/step - accuracy: 0.5419 - loss: 2.6452
Epoch 8/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 200s 514ms/step - accuracy: 0.5717 - loss: 2.4813
Epoch 9/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 200s 508ms/step - accuracy: 0.6023 - loss: 2.3179
Epoch 10/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 206s 523ms/step - accuracy: 0.6292 - loss: 2.1643


In [ ]:
model.save("text_gen_model2.h5")
with open("history2.p", "wb") as f:
    pickle.dump(history, f)

In [ ]:
model = load_model("text_gen_model2.h5")
history = pickle.load(open("history2.p", "rb"))

In [ ]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [ ]:
possible = predict_next_word("I will have to look into this thing because I", 5)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
for idx in possible:
    print(unique_tokens[idx])

give
re
went
saw
ll


In [ ]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [ ]:
generate_text("I will have to look into this thing because I", 100, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

'I will have to look into this thing because I went anything if she isn to being particularly victories under women changes said one black put already found cruz nan not a private crowd and i ll say i don might do all and so things can t like my clear which president street cruz lost him or this was called and among his wages away claims and caused some t can passed today data economic votes showing all interview ago included your abroad scandal s healthcare bolduan white claims reported trump claims of all some on trump would efforts a series for quickly 4 within but authorities by d'

In [ ]:
generate_text("The president of the United States announced yesterday that he", 100, 10)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━

'The president of the United States announced yesterday that he offering all who knew up in mind it seems out the process including all interview however january n roosevelt congress supported as its personal had accused they failed known at 4 tv crisis on t 25 federal bearer in chief nan nan and many leaders here far sees said 45 others are actually killed as never call up with ayn at our federal bearer providing an latest divided away very able that has no much problem we know being demand at it with still since their hampshire and true among his current raising their polls nan in the front deadly'

In [ ]:
for idx in predict_next_word("The president will most likely not be there to help", 5):
    print(unique_tokens[idx])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
want
have
were
because
say
